This notebook uses the OpenStudio Meta-CLI to submit OSA jobs to OpenStudio-Server.  

The OpenStudio Meta-CLI is a ruby executable that can parse OpenStudio Analysis (OSA) json files and submit the appropriate job type and payload to a running OpenStudio-Server. These files can describe complex analyses, such as building-stock analysis, design optimization, uncertainty quantification, and calibration.  This example requires a running server.

In [1]:
RUBY_CMD = 'C:\ParametricAnalysisTool-3.3.0\pat\ruby\bin\ruby'
#META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
META_CLI = File.absolute_path('C:\ParametricAnalysisTool-3.3.0\pat\OpenStudio-server\bin\openstudio_meta')
PROJECT = Dir.pwd
HOST = 'localhost'

@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD

"C:\\ParametricAnalysisTool-3.3.0\\pat\\ruby\\bin\\ruby"

In [4]:
command = "#{@ruby_cmd} #{@meta_cli} run_analysis --debug --verbose '#{@project}/analysis.json' 'http://#{@host}' -z 'analysis' -a single_run"

"C:\\ParametricAnalysisTool-3.3.0\\pat\\ruby\\bin\\ruby C:/ParametricAnalysisTool-3.3.0/pat/OpenStudio-server/bin/openstudio_meta run_analysis --debug --verbose 'C:/Projects/Notebooks/osw_project/analysis.json' 'http://localhost' -z 'analysis' -a single_run"

In [5]:
run_analysis = system(command)

true

In [6]:
sleep 5

5

In [7]:
require 'rest-client'
require 'json'
a = RestClient.get "http://#{@host}/analyses.json"
a = JSON.parse(a, symbolize_names: true)
a = a.sort { |x, y| x[:created_at] <=> y[:created_at] }.reverse
analysis = a[0]
analysis_id = analysis[:_id]

"16f16f84-d96d-49db-83ee-c09dde6c5673"

In [8]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:status]

"started"

In [9]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:problem][:analysis_type]

"single_run"

In [19]:
# get all data points in this analysis
a = RestClient.get "http://#{@host}/data_points.json"
a = JSON.parse(a, symbolize_names: true)
data_points = []
a.each do |data_point|
  if data_point[:analysis_id] == analysis_id
    data_points << data_point
  end
end
data_points.size

1

In [20]:
data_point_id = data_points[0][:_id]

"9ea9754b-47e5-49e9-bcb7-cf0c0b10df1c"

In [21]:
dp = RestClient.get "http://#{@host}/data_points/#{data_point_id}.json"
dp = JSON.parse(dp, symbolize_names: true)
data_points_status = dp[:data_point][:status]

"completed"

In [22]:
dp[:data_point][:status_message]

"completed normal"

In [25]:
results = dp[:data_point][:results][:calibration_reports_enhanced]

{:electric_nmbe_limit=>5.0, :electric_cvrmse_limit=>15.0, :gas_nmbe_limit=>5.0, :gas_cvrmse_limit=>15.0, :fuel_oil_nmbe_limit=>5.0, :fuel_oil_cvrmse_limit=>15.0, :ashrae_max_nmbe=>5.0, :ashrae_max_cvrmse=>15.0, :femp_max_nmbe=>15.0, :femp_max_cvrmse=>10.0, :electricity_consumption_cvrmse=>0.4545908511694533, :electricity_consumption_nmbe=>-0.43459090244563026, :electricity_period_1_start_date=>"2013-Jan-01", :electricity_period_1_end_date=>"2013-Jan-31", :electricity_period_1_consumption_actual=>2392.55, :electricity_period_1_consumption_modeled=>2396.9374669288045, :electricity_period_1_peak_demand_modeled=>8.07783303221987, :electricity_period_1_consumption_nmbe=>0.18338036525064533, :electricity_period_2_start_date=>"2013-Feb-01", :electricity_period_2_end_date=>"2013-Feb-28", :electricity_period_2_consumption_actual=>2152.71, :electricity_period_2_consumption_modeled=>2156.9649937436, :electricity_period_2_peak_demand_modeled=>8.63936588113086, :electricity_period_2_consumption_nmb

In [26]:
results.slice(:electricity_consumption_cvrmse, :electricity_consumption_nmbe, :natural_gas_consumption_cvrmse, :natural_gas_consumption_nmbe)

{:electricity_consumption_cvrmse=>0.4545908511694533, :electricity_consumption_nmbe=>-0.43459090244563026, :natural_gas_consumption_cvrmse=>1.4828909643126638, :natural_gas_consumption_nmbe=>1.0767346762992567}

In [27]:
objectives_json = RestClient.get "http://#{@host}/data_points/#{data_point_id}/download_result_file?filename=objectives.json"
objectives_json = JSON.parse(objectives_json, symbolize_names: true)

{:objective_function_1=>0.4545908511694533, :objective_function_group_1=>1.0, :objective_function_2=>-0.43459090244563026, :objective_function_group_2=>2.0, :objective_function_3=>1.4828909643126638, :objective_function_group_3=>3.0, :objective_function_4=>1.0767346762992567, :objective_function_group_4=>4.0}

In [28]:
require 'open-uri'
File.write 'results.csv', URI.open("http://#{@host}/analyses/#{analysis_id}/download_data.csv?export=true").read


690